In [2]:
import gymnasium as gym
import os
import sys
sys.path.append("./jsbgym")
import jsbgym
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack

In [2]:
env = gym.make("A320-HeadingControlTask-Shaping.STANDARD-NoFG-v0", render_mode="human")

In [3]:
env.reset()



     JSBSim Flight Dynamics Model v1.2.3 Jun  7 2025 19:28:34
            [JSBSim-ML v2.0]

JSBSim startup beginning ...



(array([ 5.00000000e+03,  1.21430643e-17,  1.50920942e-16,  4.21950000e+02,
         1.06581410e-14, -1.06581410e-14,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -3.72529030e-09,  0.00000000e+00]),
 {})

In [4]:
for episode in range(1, 6):
    obs, info = env.reset()
    done = False
    total_reward = 0
    while not done:
        obs, reward, done, _, info = env.step(env.action_space.sample())
        total_reward += reward
    print("Total Reward for episode {} is {}".format(episode, total_reward))

Total Reward for episode 1 is 36.52024146952639
Total Reward for episode 2 is 37.711868311524626
Total Reward for episode 3 is 29.90353997933902
Total Reward for episode 4 is 32.42956182570246
Total Reward for episode 5 is 30.81420349048216


In [5]:
class TrainAndLoggingCallback(BaseCallback):
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(
                self.save_path, "best_model_{}".format(self.n_calls)
            )
            self.model.save(model_path)

        return True

In [6]:
CHECKPOINT_DIR = "./train/"
LOG_DIR = "./logs/"

In [7]:
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [8]:
model = PPO("MlpPolicy", env, tensorboard_log=LOG_DIR, device="cpu")

In [48]:
model.learn(total_timesteps=10000000, callback=callback)
model.save("JSBSim_10000000_steps")

KeyboardInterrupt: 

## Visualization

In [14]:
env = gym.make("PA28-HeadingControlTask-Shaping.STANDARD-FG-v0", render_mode="flightgear")
env.reset()
# model = PPO.load(os.path.join("train", "saved_models", "b747_heading_3000000"), device="cpu")
model = PPO.load(os.path.join("train", "best_model_9034000"), device="cpu")



     JSBSim Flight Dynamics Model v1.2.3 Jun  7 2025 19:28:34
            [JSBSim-ML v2.0]

JSBSim startup beginning ...



In [15]:
env.render()

jaaaa


In [16]:
for episode in range(1, 6):
    obs, _ = env.reset()
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, _, info = env.step(action)
        total_reward += reward
    print("Total Reward for episode {} is {}".format(episode, total_reward))

/home/pietbcm/diff_thrust_rl/jsbgym/jsbgym/environment.py:131: UserWarning: If training, use NoFG instead of FG in the env_id. Using FG will cause errors while training after a while.
  warnings.warn(


ValueError: Error: Unexpected observation shape (11,) for Box environment, please use (17,) or (n_env, 17) for the observation shape.

In [17]:
env.close()